In [18]:
percorso_base = '2014.01.21 - Gara Carbonera'
fasce = ['A', 'B', 'C', 'D', 'E']
categorie = ['BIANCA BI-GI', 'GIALLA GI-AR', 'ARANCIO AR-VE', 'VERDE VE-BLU', 'BLU BLU-MAR', 'MARRONE E NERA MASCHILE', 'MARRONE E NERA FEMMINILE']

import pandas as pd
import xlwings as xw
import os
import shutil
import random

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)

# Carica il file con tutti gli iscritti
df = pd.read_excel('2014.01.21 - Gara Carbonera/ISCRITTI 1 TROFEO CARBONERA 2024.xlsx')

# Converte tutto in maiuscolo
df = df.map(lambda x: x.upper() if isinstance(x, str) else x)

# Elimina ' dai nomi di campo per comodità
df.columns = [col.replace("'", "").replace('"', '') for col in df.columns]

# Uniforma i dati scritti male
df['CINTURA'] = df['CINTURA'].replace(to_replace='VERDE VE-BL', value='VERDE VE-BLU', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='MARRONE NERA', value='MARRONE E NERA', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='GIALLA GI-ARANCIO', value='GIALLA GI-AR', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='BLU BL-MA', value='BLU BLU-MAR', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='BIANCA E BI-GI', value='BIANCA BI-GI', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='ARANCIO A-V', value='ARANCIO AR-VE', regex=False)
df['SOCIETA'] = df['SOCIETA'].replace(to_replace='KARATE SACILE FONTANAFREDDA\xa0', value='KARATE SACILE FONTANAFREDDA', regex=False)

# Suddivide le marroni e nere in categorie distinte in base al sesso
df.loc[(df['CINTURA'] == 'MARRONE E NERA') & (df['M / F'] == 'M'), 'CINTURA'] = 'MARRONE E NERA MASCHILE'
df.loc[(df['CINTURA'] == 'MARRONE E NERA') & (df['M / F'] == 'F'), 'CINTURA'] = 'MARRONE E NERA FEMMINILE'

# Elimina colonne inutili
df.drop(columns=['ISCRITTO'], inplace=True)

# Crea una colonna col numero identificativo dell'atleta
df.insert(0, 'ID_ATLETA', range(1, len(df) + 1))

# Crea un indice univoco ai soli fini dell'elaborazione
df.reset_index(drop=True, inplace=True)

c:\Users\932197\Documents\dati_locali\git-hub\karate\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [19]:
df['CINTURA'].unique()

array(['VERDE VE-BLU', 'GIALLA GI-AR', 'ARANCIO AR-VE', 'BLU BLU-MAR',
       'MARRONE E NERA MASCHILE', 'BIANCA BI-GI',
       'MARRONE E NERA FEMMINILE'], dtype=object)

In [20]:
df['SOCIETA'].unique()

array(['SCUOLA KARATE MIGNAGOLA', 'ASD KARATE TREVIGNANO E MONTEBELLUNA',
       'KARATE SACILE FONTANAFREDDA', 'KARATE KORIYAMA',
       'KI KAI DOJO CARBONERA', 'SCUOLA KARATE RESANA',
       'KARATE BADOERE E MORGANO', 'KS GALLIERA E ROSSANO',
       'KARATE CASTELFRANCO', 'KARATE SHINGITAIKAN RAGOGNA'], dtype=object)

In [21]:
df.head()

,ID_ATLETA,COGNOME,NOME,ANNO,M / F,CINTURA,SOCIETA,QUOTA DI ISCRIZIONE,FASCIA
0,1,PIERAGNOLO,LEONE,2015,M,VERDE VE-BLU,SCUOLA KARATE MIGNAGOLA,NaN,FASCIA C
1,2,RONCATO,ELISA,2014,F,GIALLA GI-AR,SCUOLA KARATE MIGNAGOLA,12.0,FASCIA C
2,3,STEINER,FEDERICO,2015,M,ARANCIO AR-VE,SCUOLA KARATE MIGNAGOLA,12.0,FASCIA C
3,4,BUCCI,GIOIA,2012,F,GIALLA GI-AR,SCUOLA KARATE MIGNAGOLA,12.0,FASCIA D
4,5,BUCCI,FRANCESCO,2016,M,GIALLA GI-AR,SCUOLA KARATE MIGNAGOLA,12.0,FASCIA B


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID_ATLETA            159 non-null    int64  
 1   COGNOME              159 non-null    object 
 2   NOME                 159 non-null    object 
 3   ANNO                 159 non-null    int64  
 4   M / F                159 non-null    object 
 5   CINTURA              159 non-null    object 
 6   SOCIETA              159 non-null    object 
 7   QUOTA DI ISCRIZIONE  158 non-null    float64
 8   FASCIA               159 non-null    object 
dtypes: float64(1), int64(2), object(6)
memory usage: 11.3+ KB


In [23]:
# Prepara le cartelle che conterranno i tabelloni generati. Se esistenti vengono cancellate e ricreate
def ricrea_cartella_tabelloni():
    if os.path.exists(f'{percorso_base}/Tabelloni generati'):
        shutil.rmtree(f'{percorso_base}/Tabelloni generati')
    os.makedirs(f'{percorso_base}/Tabelloni generati')   
    os.makedirs(f'{percorso_base}/Tabelloni generati/PDF')   
    os.makedirs(f'{percorso_base}/Tabelloni generati/Excel')   
    os.makedirs(f'{percorso_base}/Tabelloni generati/parquet')  

# Restituisce gli atleti che partecipano a una determinata fascia e categoria. Se gli atleti sono dispari restituisce anche l'atleta appartenente 
# alla società con il maggior numero di iscritti per quella fascia/categoria
def iscritti_per_tabellone(fascia, categoria):
    iscritti = df[(df['FASCIA'] == f'FASCIA {fascia}') & (df['CINTURA'] == categoria)].copy()
    n_iscritti = len(iscritti)
    atleta_singolo = None
    if n_iscritti % 2 == 1:
        # Se sono DISPARI, sceglie l'atleta che non farà il primo incontro fra quelli della società col maggior numero di atleti iscritti
        soc_maggiori_iscritti = iscritti.groupby(['SOCIETA'])[['SOCIETA']].value_counts().idxmax()
        id_atleta_singolo = iscritti[iscritti['SOCIETA'] == soc_maggiori_iscritti].index[0]
        atleta_singolo = iscritti.loc[id_atleta_singolo]
        iscritti.drop(id_atleta_singolo, inplace=True)

    iscritti['ISCRITTI_PER_SOCIETA'] = iscritti.groupby('SOCIETA')['SOCIETA'].transform('count')
    iscritti = iscritti.sort_values(by='ISCRITTI_PER_SOCIETA', ascending=False)
    
    return iscritti, atleta_singolo

# Modifiche manuali ai tabelloni: scambia di posto due iscritti in un tabellone
def scambia_iscritti(tab, n_incontro_1, cintura_r_b_1, n_incontro_2, cintura_r_b_2):
    # Crea maschere per le righe da scambiare
    mask1 = (tab['N_INCONTRO'] == n_incontro_1) & (tab['CINTURA'] == cintura_r_b_1)
    mask2 = (tab['N_INCONTRO'] == n_incontro_2) & (tab['CINTURA'] == cintura_r_b_2)

    # Verifica se entrambe le righe esistono
    if mask1.any() and mask2.any():
        # Copia temporaneamente una delle righe
        riga_temp = tab.loc[mask1, ['SOCIETA', 'COGNOME', 'NOME']].copy()

        # Scambia le righe
        tab.loc[mask1, ['SOCIETA', 'COGNOME', 'NOME']] = tab.loc[mask2, ['SOCIETA', 'COGNOME', 'NOME']].values
        tab.loc[mask2, ['SOCIETA', 'COGNOME', 'NOME']] = riga_temp.values

    return tab

# Modifiche manuali ai tabelloni: elimina un iscritto da un tabellone
def elimina_iscritto(tab, n_incontro, cintura_r_b):
    # Crea maschera per trova l'atleta
    mask = (tab['N_INCONTRO'] == n_incontro) & (tab['CINTURA'] == cintura_r_b)  

    # Verifica se la riga esiste
    if mask.any():
        tab = tab.drop(tab[mask].index)
    
    return tab

# Carica un tabellone dalla raccolta di file parquet
def carica_tabellone(n_tab):
    nome_file = None
    tab = None
    for file in os.listdir(f'{percorso_base}/Tabelloni generati/parquet'):
        if file.startswith(str(n_tab).zfill(2)):
            nome_file = file
            break
    if nome_file:
        path = f'{percorso_base}/Tabelloni generati/parquet/{nome_file}'
        tab = pd.read_parquet(path)
    return tab

# Crea il tabellone in excel e lo salva in pdf e parquet
def salva_tabellone(tab, n_tab):
    fascia = tab['FASCIA'].iloc[0]
    categoria = tab['CATEGORIA'].iloc[0]
    tab_vuoto = f'{percorso_base}/Tabellone vuoto.xlsx'
    foglio = 'TAB.SINGOLI'
    nome_file = f'{str(n_tab).zfill(2)}) FASCIA {fascia} - {categoria}'
    riga_inizio_tabella = 8
    riga_fine_tabella = 117
    
    app = xw.App(visible=False)
    wb = app.books.open(tab_vuoto)
    sheet = wb.sheets[foglio]  
    sheet['C1'].value = fascia
    sheet['F1'].value = categoria
    sheet[f'A{riga_inizio_tabella}'].value = tab.drop(columns=['FASCIA', 'CATEGORIA']).values
    sheet['J1'].value = n_tab
    prima_riga_vuota = riga_inizio_tabella
    for riga in range(riga_inizio_tabella, riga_fine_tabella):
        if (not sheet[f'C{riga}'].value) and (sheet[f'B{riga}'].value == 'R'):
            prima_riga_vuota = riga
            break
    rg = sheet.range(f'{prima_riga_vuota}:{riga_fine_tabella}')
    rg.delete(shift='up')
    sheet.page_setup
    sheet.to_pdf(f'{percorso_base}/Tabelloni generati/PDF/{nome_file}.pdf')
    wb.save(f'{percorso_base}/Tabelloni generati/Excel/{nome_file}.xlsx')
    tab.to_parquet(f'{percorso_base}/Tabelloni generati/parquet/{nome_file}.parquet')
    wb.close()
    app.quit()

def genera_tabelloni():
    ricrea_cartella_tabelloni() # Crea la struttura delle cartelle per i tabelloni generati
    tot_tabelloni = len(fasce) * len(categorie)
    n_tab = 1 # Variabile che serve solo durante la generazione per vedere quale tabellone sta elaborando, anche se poi non verrà salvato perché vuoto
    n_tab_salvati = 1 # Variabile che corrisponde al numero di tabellone 
    for fascia in fasce:
        for categoria in categorie:
            print(f'Tabellone n.{n_tab} di {tot_tabelloni}: Fascia {fascia}, Categoria: {categoria}')
            iscritti, atleta_singolo = iscritti_per_tabellone(fascia, categoria)
            tot_incontri = len(iscritti) // 2
            accoppiati = [] # Variabile che tiene traccia degli atleti già inseriti in un incontro
            riga = {} # riga del tabellone dataframe
            tab = pd.DataFrame(columns=['N_INCONTRO', 'CINTURA', 'SOCIETA', 'SOC2', 'SOC3', 'COGNOME', 'NOME', 'FASCIA', 'CATEGORIA'])
            n_incontro = 0
            if (not iscritti.empty) or (atleta_singolo is not None):
                for n_incontro in range(1, tot_incontri+1):
                    
                    # Trova il primo partecipante all'incontro
                    for i in range(len(iscritti)):
                        atleta1 = iscritti.iloc[i]
                        if atleta1.name not in accoppiati:
                            accoppiati.append(atleta1.name)
                            break
                        
                    # Trova il secondo fra quelli di società diverse dal primo partecipante
                    df_atleta2 = iscritti.loc[(iscritti['SOCIETA'] != atleta1['SOCIETA'])].sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1)
                    trovato = False
                    for i in range(len(df_atleta2)):
                        atleta2 = df_atleta2.iloc[i]
                        if atleta2.name not in accoppiati:
                            accoppiati.append(atleta2.name)
                            trovato = True
                            break     
                    
                    # Nel caso non ci fossero più società diverse, accoppia due atleti della stessa società nell'incontro   
                    if not trovato:
                        df_atleta2 = iscritti.loc[(iscritti['SOCIETA'] == atleta1['SOCIETA'])].sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1)
                        for i in range(len(df_atleta2)):
                            atleta2 = df_atleta2.iloc[i]
                            if atleta2.name not in accoppiati:
                                accoppiati.append(atleta2.name)
                                break       
                    
                    # Aggiunge i due atleti al tabellone 
                    tab.loc[len(tab)] = [n_incontro, 'R', atleta1['SOCIETA'], '', '', atleta1['COGNOME'], atleta1['NOME'], fascia, categoria]
                    tab.loc[len(tab)] = [n_incontro, 'B', atleta2['SOCIETA'], '', '', atleta2['COGNOME'], atleta2['NOME'], fascia, categoria]
                
                # Mescola l'ordine degli incontri
                if not iscritti.empty: # Se c'è un unico iscritto non serve mescolare
                    pairs = [tab.iloc[i:i+2] for i in range(0, len(tab), 2)]
                    random.shuffle(pairs)
                    random.shuffle(pairs)
                    random.shuffle(pairs)
                    random.shuffle(pairs)
                    random.shuffle(pairs)
                    tab = pd.concat(pairs).reset_index(drop=True)
                    n = 1
                    for i in range(0, len(tab), 2):
                        tab.loc[i, 'N_INCONTRO'] = n
                        tab.loc[i+1, 'N_INCONTRO'] = n
                        n += 1

                # Se il numero di atleti è dispari aggiunge alla fine l'atleta della società col numero maggiore di partecipanti
                if atleta_singolo is not None:
                    n_incontro += 1
                    tab.loc[len(tab)] = [n_incontro, 'R', atleta_singolo['SOCIETA'], '', '', atleta_singolo['COGNOME'], atleta_singolo['NOME'], fascia, categoria]
                
                # Salva il tabellone nei vari formati
                salva_tabellone(tab, n_tab_salvati)
                n_tab_salvati += 1
                
            n_tab += 1


### GENERA TABELLONI
genera_tabelloni()

In [24]:
### Modifiche manuali
n_tabellone = 1

tab = carica_tabellone(n_tabellone)
tab_prima = tab.copy()

#tab = scambia_iscritti(tab, 2, 'B', 3, 'R')
tab = elimina_iscritto(tab, 3, 'R')

salva_tabellone(tab, n_tabellone)
tab_dopo = tab.copy()

In [25]:
##### PRIMA
tab_prima

,N_INCONTRO,CINTURA,SOCIETA,SOC2,SOC3,COGNOME,NOME,FASCIA,CATEGORIA
0,1,R,SCUOLA KARATE MIGNAGOLA,,,AVDULAJ,ALESSIO,A,BIANCA BI-GI
1,1,B,KI KAI DOJO CARBONERA,,,ZARATTINI,LEONARDO,A,BIANCA BI-GI
2,2,R,KS GALLIERA E ROSSANO,,,HUZUM,LUCA,A,BIANCA BI-GI
3,2,B,KARATE SHINGITAIKAN RAGOGNA,,,DELLE CASE,SAMUELE,A,BIANCA BI-GI
4,3,R,KARATE SACILE FONTANAFREDDA,,,DEL COL,ATHENA,A,BIANCA BI-GI


In [26]:
##### DOPO
tab_dopo

,N_INCONTRO,CINTURA,SOCIETA,SOC2,SOC3,COGNOME,NOME,FASCIA,CATEGORIA
0,1,R,SCUOLA KARATE MIGNAGOLA,,,AVDULAJ,ALESSIO,A,BIANCA BI-GI
1,1,B,KI KAI DOJO CARBONERA,,,ZARATTINI,LEONARDO,A,BIANCA BI-GI
2,2,R,KS GALLIERA E ROSSANO,,,HUZUM,LUCA,A,BIANCA BI-GI
3,2,B,KARATE SHINGITAIKAN RAGOGNA,,,DELLE CASE,SAMUELE,A,BIANCA BI-GI
